## [데이콘] 와인품질분류
- https://dacon.io/competitions/open/235610/overview/description

# Baseline

In [57]:
import pandas as pd
from google.colab import drive, files
drive.mount('/content/drive')
import warnings
warnings.filterwarnings(action='ignore')
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

train = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/train.csv')
test = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/test.csv')
sub = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/sample_submission.csv')


# 변주형 범수 처리
train['type'] = train['type'].map(lambda x : 1 if x == 'white' else 0)
test['type'] = test['type'].map(lambda x : 1 if x == 'white' else 0)

# 불필요 컬럼 삭제
label = train['quality']
train.drop(['index', 'quality'], axis=1, inplace=True)
test.drop('index', axis=1, inplace=True)


# 피쳐 엔지니어링 (정제, 이상치처리, 파생변수 생성)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

train = sc.fit_transform(train)
test = sc.transform(test)


# 모델링
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=2022, n_estimators=200)

# 교차검증
score = cross_val_score(model, train, label, cv=5, scoring='accuracy')

print(score)
print(score.mean())


# 제출
model.fit(train, label)
preds = model.predict(test)
sub.iloc[:,1] = preds
sub.to_csv('wine.csv', index=False)
files.download('wine.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[0.68363636 0.68727273 0.67060965 0.66606005 0.6633303 ]
0.6741818181818182


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model Selection

In [58]:
!pip install catboost

In [59]:
from sklearn.linear_model import RidgeClassifier, LogisticRegression  
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

model_list = [RidgeClassifier(), LogisticRegression(), RandomForestClassifier(random_state=2021), GradientBoostingClassifier(), AdaBoostClassifier(), XGBClassifier(), LGBMClassifier(), CatBoostClassifier(verbose=0)]
scores = []

for model in model_list:
  score = cross_val_score(model, train, label, cv=5, scoring='accuracy')
  scores.append(score.mean())


model_test = pd.DataFrame({'model':model_list, 'score':scores})
model_test.sort_values('score')

,model,score
4,"AdaBoostClassifier(algorithm='SAMME.R', base_e...",0.402040
0,"RidgeClassifier(alpha=1.0, class_weight=None, ...",0.531925
1,"LogisticRegression(C=1.0, class_weight=None, d...",0.545205
5,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.578679
3,"GradientBoostingClassifier(ccp_alpha=0.0, crit...",0.585773
6,"LGBMClassifier(boosting_type='gbdt', class_wei...",0.633619
7,<catboost.core.CatBoostClassifier object at 0x...,0.649624
2,"RandomForestClassifier(bootstrap=True, ccp_alp...",0.675455
